# Reticle assembly

With gdsfactory you can easily go from a Component, to a Component sweep, to a top level Component full of subcomponents.

## Test protocols

To measure your reticle / die after fabrication you need to decide your test configurations. This includes things like:

- `Individual input and output fibers` versus `fiber array`. I reccommend `fiber array` for easier testing and higher throughtput, but also understand the flexibility of single fibers for some cases.
- Fiber array pitch (127um or 250um) if using a fiber array.
- Pad pitch for DC and RF high speed probes (100, 125, 150, 200um). Probe configuration (GSG, GS ...)
- Test layout for DC, RF and optical fibers.


To enable automatic testing you need to add labels to all the devices that you want to test. GDS labels are not fabricated and are only visible in the GDS file.

Lets show some different automatic labeling schemas:

1. SiEPIC ubc Ebeam PDK schema, labels one of the grating couplers from the fiber array.
2. `gf.dft.add_label_yaml` includes a YAML based Label with all testing information.
3. EHVA automatic testers, include a Label component declaration as described in this [doc](https://drive.google.com/file/d/1kbQNrVLzPbefh3by7g2s865bcsA2vl5l/view)


### 1. SiEPIC labels

In [ ]:
import gdsfactory as gf

gf.config.set_plot_options(show_subports=False)

In [ ]:
mmi = gf.components.mmi2x2()
mmi_te_siepic = gf.dft.add_fiber_array_siepic(component=mmi)
mmi_te_siepic

In [ ]:
mmi_te_siepic.get_labels(depth=1)

### 2. YAML labels

In [ ]:
mmi = gf.components.mmi2x2()
mmi_te_yaml = gf.routing.add_fiber_array(mmi, get_input_labels_function=None)
gf.dft.add_label_yaml(component=mmi_te_yaml)
mmi_te_yaml

In [ ]:
mmi_te_yaml.get_labels(depth=0)

### 3. EHVA labels

In [ ]:
mmi_te_ehva = gf.routing.add_fiber_array(mmi, get_input_labels_function=None)
mmi_te_ehva.remove_labels(lambda x: True)
gf.dft.add_label_ehva(component=mmi_te_ehva, die="mpw1")
mmi_te_ehva

In [ ]:
mmi_te_ehva.get_labels(depth=0)

One advantage of the YAML and EHVA formats is that you can track any changes on the components directly from the GDS label, as the label already stores any changes of the child device, as well as any settings that you specify.

Settings can have many levels of hierarchy, but you can still access any children setting with `:` notation.


```
grating_coupler:
    function: grating_coupler_elliptical_trenches
    settings:
        polarization: te
        taper_angle: 35

```

In [ ]:
mmi = gf.components.mmi2x2(length_mmi=10)
mmi_te_ehva = gf.routing.add_fiber_array(mmi, get_input_labels_function=None)
mmi_te_ehva.remove_labels(lambda x: True)
gf.dft.add_label_ehva(
    component=mmi_te_ehva,
    die="mpw1",
    metadata_include_parent=["grating_coupler:settings:polarization"],
)
mmi_te_ehva

In [ ]:
mmi_te_ehva.get_labels(depth=0)

## Pack

Lets start with a resistance sweep, where you change the resistance width to measure sheet resistance.

In [ ]:

sweep = [gf.components.resistance_sheet(width=width) for width in [1, 10, 100]]
m = gf.pack(sweep)
c = m[0]
c

Then we add spirals with different lengths to measure waveguide propagation loss.

In [ ]:
spiral = gf.components.spiral_inner_io_fiber_single()
spiral

In [ ]:
spiral_te = gf.routing.add_fiber_single(
    gf.functions.rotate(gf.components.spiral_inner_io_fiber_single, 90)
)
spiral_te

In [ ]:
# which is equivalent to
spiral_te = gf.compose(
    gf.routing.add_fiber_single,
    gf.functions.rotate90,
    gf.components.spiral_inner_io_fiber_single,
)
c = spiral_te(length=10e3)
c

In [ ]:
import gdsfactory as gf

add_label_ehva_mpw1 = gf.partial(gf.dft.add_label_ehva, die="mpw1")
add_fiber_single_no_labels = gf.partial(
    gf.routing.add_fiber_single, get_input_label_text_function=None
)

spiral_te = gf.compose(
    add_label_ehva_mpw1,
    add_fiber_single_no_labels,
    gf.functions.rotate90,
    gf.components.spiral_inner_io_fiber_single,
)
sweep = [spiral_te(length=length) for length in [10e3, 20e3, 30e3]]
m = gf.pack(sweep)
c = m[0]
c

In [ ]:
c.get_labels()

Together with GDS labels that are not fabricated, you can also add some physical labels that will be fabricated.

For example you can add prefix `S` at the `north-center` of each spiral using `text_rectangular` which is DRC clean and anchored on `nc` (north-center)

In [ ]:
text_metal3 = gf.partial(
    gf.components.text_rectangular_multi_layer, layers=(gf.LAYER.M3,)
)

m = gf.pack(sweep, text=text_metal3, text_anchors=("nc",), text_prefix="s")
c = m[0]
c

In [ ]:
text_metal2 = gf.partial(gf.components.text, layer=gf.LAYER.M2)

m = gf.pack(sweep, text=text_metal2, text_anchors=("nc",), text_prefix="s")
c = m[0]
c

## Grid

You can also pack components with a constant spacing.

In [ ]:
g = gf.grid(sweep)
g

In [ ]:
gh = gf.grid(sweep, shape=(1, len(sweep)))
gh

In [ ]:
gh_ymin = gf.grid(sweep, shape=(1, len(sweep)), align_y="ymin")
gh_ymin

You can also add text labels to each element of the sweep

In [ ]:
gh_ymin = gf.grid_with_text(
    sweep, shape=(1, len(sweep)), align_y="ymin", text=text_metal3
)
gh_ymin

You can modify the text by customizing the `text_function` that you pass to `grid_with_text`

In [ ]:
gh_ymin_m2 = gf.grid_with_text(
    sweep, shape=(1, len(sweep)), align_y="ymin", text=text_metal2
)
gh_ymin_m2

You have 2 ways of defining a mask:

1. in python
2. in YAML


## Component in python

You can define a Component top cell reticle or die using `grid` and `pack` python functions.

In [ ]:
import gdsfactory as gf

text_metal3 = gf.partial(
    gf.components.text_rectangular_multi_layer, layers=(gf.LAYER.M3,)
)
grid = gf.partial(gf.grid_with_text, text=text_metal3)
pack = gf.partial(gf.pack, text=text_metal3)

gratings_sweep = [
    gf.components.grating_coupler_elliptical(taper_angle=taper_angle)
    for taper_angle in [20, 30, 40]
]
gratings = grid(gratings_sweep, text=None)
gratings

In [ ]:
gratings_sweep = [
    gf.components.grating_coupler_elliptical(taper_angle=taper_angle)
    for taper_angle in [20, 30, 40]
]
gratings_loss_sweep = [
    gf.components.grating_coupler_loss_fiber_single(grating_coupler=grating)
    for grating in gratings_sweep
]
gratings = grid(
    gratings_loss_sweep, shape=(1, len(gratings_loss_sweep)), spacing=(40, 0)
)
gratings

In [ ]:
sweep_resistance = [
    gf.components.resistance_sheet(width=width) for width in [1, 10, 100]
]
resistance = gf.pack(sweep_resistance)[0]
resistance

In [ ]:
spiral_te = gf.compose(
    gf.routing.add_fiber_single,
    gf.functions.rotate90,
    gf.components.spiral_inner_io_fiber_single,
)
sweep_spirals = [spiral_te(length=length) for length in [10e3, 20e3, 30e3]]
spirals = gf.pack(sweep_spirals)[0]
spirals

In [ ]:
mask = gf.pack([spirals, resistance, gratings])[0]
mask

As you can see you can define your mask in a single line.

For more complex mask, you can also create a new cell to build up more complexity

In [ ]:
@gf.cell
def mask():
    c = gf.Component()
    c << gf.pack([spirals, resistance, gratings])[0]
    c << gf.components.seal_ring(c.bbox)
    return c


c = mask(cache=False)
c

## Metadata

When saving GDS files is also convenient to store the metadata settings that you used to generate the GDS file.

In [ ]:
gdspath = c.write_gds_with_metadata(gdsdir="extra")

In [ ]:
yaml_path = gdspath.with_suffix(".yml")

In [ ]:
labels_path = gf.mask.write_labels(gdspath=gdspath, layer_label=(201, 0))

In [ ]:
from omegaconf import OmegaConf

mask_metadata = OmegaConf.load(yaml_path)

In [ ]:
test_metadata = tm = gf.mask.merge_test_metadata(
    labels_path=labels_path, mask_metadata=mask_metadata
)

In [ ]:
tm.keys()

```

CSV labels  ------|
                  |--> merge_test_metadata dict
                  |
YAML metatada  ---

```

In [ ]:
spiral_names = [s for s in test_metadata.keys() if s.startswith("spiral")]
spiral_names

In [ ]:
spiral_lengths = [
    test_metadata[spiral_name].info.length for spiral_name in spiral_names
]
spiral_lengths

In [ ]:
gc_names = [s for s in test_metadata.keys() if s.startswith("grating")]
gc_names

In [ ]:
gc_taper_angles = [test_metadata[name].full.taper_angle for name in gc_names]
gc_taper_angles

## Component in YAML

You can also define your component in YAML format thanks to `gdsfactory.read.from_yaml`

You need to define:

- instances
- placements

and you can leverage:

1. `pack_doe`
2. `pack_doe_grid`

### 1. pack_doe

`pack_doe` places components as compact as possible

When running this tutorial make sure you UNCOMMENT this line `%matplotlib widget` so you can live update your changes in the YAML file

`# %matplotlib widget`  -> `%matplotlib widget`

In [ ]:
# %matplotlib widget

import ipywidgets
from IPython.display import clear_output
import matplotlib.pyplot as plt
import gdsfactory as gf

x = ipywidgets.Textarea(rows=20, columns=480)

x.value = """
name: mask_grid

instances:
  rings:
    component: pack_doe
    settings:
      doe: ring_single
      settings:
        radius: [30, 50, 20, 40]
        length_x: [1, 2, 3]
      do_permutations: True
      function:
        function: add_fiber_array
        settings:
            fanout_length: 200

  mzis:
    component: pack_doe
    settings:
      doe: mzi
      settings:
        delta_length: [10, 100]
      function: add_fiber_array

placements:
  rings:
    xmin: 50

  mzis:
    xmin: rings,east
"""

out = ipywidgets.Output()
display(x, out)


def f(change, out=out):
    try:
        c = gf.read.from_yaml(change["new"])
        # clear_output()
        fig = c.plot()
        c.show()
        out.clear_output()
    except Exception as e:
        out.clear_output()
        with out:
            display(e)


x.observe(f, "value")
f({"new": x.value})

### 2. pack_doe_grid

`pack_doe_grid` places each component on a regular grid

In [ ]:
x.value = """
name: mask_compact

instances:
  rings:
    component: pack_doe
    settings:
      doe: ring_single
      settings:
        radius: [30, 50, 20, 40]
        length_x: [1, 2, 3]
      do_permutations: True
      function:
        function: add_fiber_array
        settings:
            fanout_length: 200


  mzis:
    component: pack_doe_grid
    settings:
      doe: mzi
      settings:
        delta_length: [10, 100]
      do_permutations: True
      spacing: [10, 10]
      function: add_fiber_array

placements:
  rings:
    xmin: 50

  mzis:
    xmin: rings,east
"""

display(x, out)